In [204]:
import pandas as pd
import numpy as np
import config
import functions
import spotipy
import random 
from fuzzywuzzy import fuzz #  fuzzy string matching techniques to identify similar strings in the DataFrame
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from IPython.display import IFrame

# Initialize scaler 

scaler = StandardScaler()

df_billboard = pd.read_csv('billboard_hot_100_df.csv')

df_my_music_training_data = pd.read_csv('my_music_training_data.csv')
df = df_my_music_training_data

In [205]:
# Initiate model 
# Input not sensitive to case 
song = input("Please enter the name of a song. Tip: include the artist name for better results: ").lower()

# Search for the song on Spotify

# Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,
                                                           client_secret=config.client_secret))
song_result = sp.search(q=song, limit=1, type="track")
matching_track_id = song_result["tracks"]["items"][0]["id"]
print("The Spotify ID of this track is", matching_track_id)

# Embed the input track 
print("You can listen to your selected track below")
input_track_iframe = IFrame(src="https://open.spotify.com/embed/track/"+matching_track_id,
                           width="320",
                           height="80",
                           frameborder="0",
                           allowtransparency="true",
                           allow="encrypted-media")
display(input_track_iframe)

# Check if the user input is similar to any value in the DataFrame
matching_songs = df_billboard[df_billboard['songs'].apply(lambda x: fuzz.partial_ratio(song.lower(), x.lower())) > threshold]

if not matching_songs.empty:
    print("\033[91m🔥 Your song is hot! 🔥\033[0m")
    
    # Retrieve a random song from the Billboard Hot 100 as a recommendation
    if not similar_songs.empty:
        recommendation = df_billboard.sample()
        print("\033[1m👇 Here's another hot song from the Billboard Hot 100 as a recommendation:\033[0m")
        print(recommendation)
        
        recommendation_song_name = recommendation.iloc[0]['songs']  # Assuming 'songs' is the column containing the song names
        song_result = sp.search(q=recommendation_song_name, limit=1, type="track")
        matching_track_id = song_result["tracks"]["items"][0]["id"]
        print("The Spotify ID of this track is", matching_track_id)

        # Embed the input track 
        print("You can listen to the recommended track below")
        input_track_iframe = IFrame(src="https://open.spotify.com/embed/track/"+matching_track_id,
                           width="320",
                           height="80",
                           frameborder="0",
                           allowtransparency="true",
                           allow="encrypted-media")
        display(input_track_iframe)
    
else:
    print("\033[1m\033[91m😔 Your song doesn't seem to be hot.")
    audio_features = sp.audio_features(matching_track_id)[0]
    
    # Create DataFrame from dictionary
    sample_df = pd.DataFrame(audio_features, index=[0]).drop(columns=["id", "type", "uri", "track_href", "analysis_url"])

    # Transform the sample data using the fitted scaler
    sample_array_scaled = scaler.transform(sample_df)
    
    from sklearn.cluster import KMeans
    
    # Define the Kmeans 
    kmeans.fit(sample_array_scaled)

    # Predict the cluster of the track 
    
    sample_pred = kmeans.predict(sample_array_scaled)
    sample_cluster = sample_pred[0]
    sample_cluster
    
    print("We predict that your chosen song falls into cluster: ", sample_cluster)
    
    # Retrieve a random song from the same cluster in the DataFrame as a recommendation
    recommendation = df[df['cluster'] == sample_cluster].sample()
    
    # Assuming recommendation.id is a NumPy array
    recommendation_id = np.array(recommendation.id)[0]

    # Embed the recommended song
    recommendation_iframe = IFrame(src="https://open.spotify.com/embed/track/" + recommendation_id,
                                   width="320",
                                   height="80",
                                   frameborder="0",
                                   allowtransparency="true",
                                   allow="encrypted-media")
    
    # Print a message about the recommendation
    print("\033[1m\033[94m🎉 Fear not! Here's a recommendation for you:")
    display(recommendation_iframe)

Please enter the name of a song. Tip: include the artist name for better results:  not nineteen forever


The Spotify ID of this track is 1bSkRvS1uWHI1QL0aJbvyC
You can listen to your selected track below


😔 Your song doesn't seem to be hot.


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# Can't seem to predict cluster 